In [ ]:
import ee, geetools

ee.Initialize()

In [ ]:
vatican = ee.Geometry.Point([12.4534, 41.9033]).buffer(1)
fc = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_CODE', 110))
fc

In [ ]:
image = ee.Image([0, 1, 2]).toArray()
image = image.geetools.removeZeros()
image

In [ ]:
image.getInfo()

In [ ]:
values = image.reduceRegion(ee.Reducer.first(), vatican, 1)

In [ ]:
values.getInfo()